# **COVID-19 Data Exploration and Analysis**

## <span style="font-size: 18px; font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);"><b>Introduction</b></span>

**Objective:** This project explores a global dataset on COVID-19 to uncover patterns and insights related to cases, deaths, and vaccinations across different countries. Using SQL, the analysis focuses on extracting meaningful statistics, calculating derived metrics such as death percentages, and identifying trends that highlight the pandemic's impact worldwide.

**Dataset Description:** The dataset includes detailed records for each country, with key attributes such as:

- Total cases and new cases reported daily
    
- Total deaths and new deaths
    
- Population of each country
    
- Vaccination data
    
- Other Health Information
    

**Key Goals of the Project:**

1. Analyze the spread and severity of COVID-19 across different countries.
    
2. Calculate critical metrics, such as the percentage of deaths relative to total cases.
    
3. Understand country-specific trends and factors contributing to variations in the impact of COVID-19.
    

This exploration enables a data-driven understanding of the pandemic and helps visualize its global impact.

## Basic Data Extraction

**Purpose:** This query retrieves essential information to establish a foundational understanding of COVID-19's spread. It includes:

- The country and date to track data over time.
    
- Total cases and new cases to measure the daily growth of infections.
    
- Total deaths to assess the pandemic's severity.
    
- Population as a baseline for calculating proportional statistics.

In [ ]:
SELECT country, date, total_cases, new_cases, total_deaths, population FROM CovidDeaths

## Calculating Death Percentage

**Purpose:** This query calculates the percentage of deaths relative to the total number of cases for each country and date. The use of `NULLIF(total_cases, 0)` prevents division by zero errors.

**Metric Explanation:**

- **Death Percentage:** The proportion of deaths among confirmed cases, expressed as a percentage, is a key metric for assessing the lethality of the virus in different regions.

**Insights Derived:**

1. Countries with the highest death rates relative to cases.
    
2. Time-based changes in death percentages, reflecting healthcare improvements or pandemic waves.
    
3. Insights into regional healthcare capacities and COVID-19 response effectiveness.

In [ ]:
SELECT country, date, total_cases, total_deaths, (total_deaths/ NULLIF(total_cases,0))*100 AS death_percentage FROM CovidDeaths

## Calculating Affected Percentage

**Purpose:** This query determines the percentage of a country's population affected by COVID-19. It helps measure the pandemic's penetration rate in each country.

**Insights Derived:**

1. Countries with the highest proportions of affected populations.
    
2. Trends in how infection rates have grown over time.

In [ ]:
SELECT country, date, total_cases, population, (total_cases/ NULLIF(population,0))*100 AS affected_percentage FROM CovidDeaths

## Maximum Cases and Affected Percentage by Country

**Purpose:** This query calculates the maximum total cases and the highest affected percentage for each country. It provides insights into countries that experienced the highest peaks during the pandemic.

**Insights Derived:**

1. Countries with the highest peak cases relative to population.
    
2. Variations in the pandemic’s impact by continent and population size.

In [ ]:
SELECT country, population, MAX(total_cases) AS max_cases, MAX((total_cases/ NULLIF(population,0)))*100 AS affected_percentage
FROM CovidDeaths
WHERE continent IS NOT NULL
GROUP BY country, population
ORDER BY affected_percentage DESC

## Death Statistics by Country

**Purpose:** This query identifies countries with the highest total deaths, highlighting regions most severely affected by fatalities.

**Insights Derived:**

1. Countries with the highest death tolls.
    
2. Regional patterns of high fatalities.

In [ ]:
SELECT country, MAX(total_deaths) AS max_deaths
FROM CovidDeaths
WHERE continent IS NOT NULL
GROUP BY country
ORDER BY max_deaths DESC;

## Death Percentage by Country

**Purpose:** This query identifies countries with the highest death percentages relative to population.

**Insights Derived:**

1. Most severely affected countries in terms of population impact.
    
2. Understanding the lethality of the pandemic in smaller versus larger populations.

In [ ]:
SELECT country, population, MAX(total_deaths) AS max_deaths,
       MAX((total_deaths / NULLIF(population, 0)) * 100) AS death_percentage
FROM CovidDeaths
WHERE continent IS NOT NULL
GROUP BY country, population
ORDER BY death_percentage DESC;

## Deaths by Continent

**Purpose:** This query summarizes total deaths at the continental level, providing a macro perspective on the pandemic's impact.

**Insights Derived:**

1. Continents most affected by COVID-19 fatalities.
    
2. A clearer view of global patterns.

In [ ]:
SELECT continent, MAX(total_deaths) AS max_deaths
FROM CovidDeaths
WHERE continent IS NOT NULL
GROUP BY continent
ORDER BY max_deaths DESC;

## Vaccination Analysis and Trends

**Purpose:** This query tracks cumulative vaccinations for each country, allowing for an analysis of vaccination rollouts over time.

**Insights Derived:**

1. Countries with the fastest vaccination rollouts.
    
2. Variations in vaccination trends between continents.

In [ ]:
SELECT cd.continent, cd.country, cd.date, population, new_vaccinations, 
       SUM(new_vaccinations) OVER (PARTITION BY cd.country ORDER BY cd.country, cd.date) AS RollingPeopleVaccinated
FROM CovidDeaths cd
JOIN CovidVaccinations cv
ON cd.country = cv.country AND cd.date = cv.date
WHERE cd.continent IS NOT NULL;

## Common Table Expression (CTE)

**Purpose:** The CTE organizes vaccination data by creating a rolling count of vaccinations for each country, then calculates the percentage of the population vaccinated over time.

**Insights Derived:**

1. Easy identification of countries with higher percentages of vaccinated populations.
    
2. Monitoring vaccination trends over time for each continent.

In [ ]:
WITH DailyVac(Continent, Country, Date, Population, New_Vaccinations, RollingPeopleVaccinated)
AS
(
SELECT cd.continent, cd.country, cd.date, population, new_vaccinations, 
       SUM(new_vaccinations) OVER (PARTITION BY cd.country ORDER BY cd.country, cd.date) AS RollingPeopleVaccinated
FROM CovidDeaths cd
JOIN CovidVaccinations cv
ON cd.country = cv.country
AND cd.date = cv.date
WHERE cd.continent IS NOT NULL
)
SELECT *, (RollingPeopleVaccinated / Population) * 100 AS percent_rolling FROM DailyVac;

## Population vs Vaccinated Population

**Purpose:** This query compares total vaccinations to population size, helping identify countries approaching population-wide immunity.

**Insights Derived:**

1. Countries leading in vaccination efforts.
    
2. The gap between vaccinated populations and total populations.

In [ ]:
SELECT cd.country, population, SUM(new_vaccinations) AS vaccinated_population
FROM CovidDeaths cd
JOIN CovidVaccinations cv
ON cd.country = cv.country AND cd.date = cv.date
WHERE cd.continent IS NOT NULL
GROUP BY cd.country, population
ORDER BY vaccinated_population DESC;

## Temporary Table for Vaccination Analysis

**Purpose:** The temporary table `#PopVsVac` stores country-level vaccination statistics, enabling efficient analysis of vaccination percentages. By aggregating data for each country and calculating the vaccinated population as a percentage of the total population, this table supports detailed exploration of global vaccination efforts.

**Insights Derived:**

1. Countries with the highest and lowest vaccination rates.
    
2. Insights into regions approaching population-wide immunity.
    
3. Identification of vaccination disparities between countries and continents.

In [ ]:
DROP TABLE IF EXISTS PopVsVac;

CREATE TABLE #PopVsVac (
    Country NVARCHAR(100),
    Population BIGINT,
    Vaccinated_Population DECIMAL
);

INSERT INTO #PopVsVac 
SELECT cd.country, population, SUM(new_vaccinations) AS vaccinated_population
FROM CovidDeaths cd JOIN CovidVaccinations cv
ON cd.country = cv.country AND cd.date = cv.date
WHERE cd.continent IS NOT NULL
GROUP BY cd.country, population
ORDER BY vaccinated_population DESC;

SELECT *, (vaccinated_population/population)*100 AS Percent_Vaccinated FROM #PopVsVac
ORDER BY Percent_Vaccinated

## Creating a View for Vaccination Analysis

**Purpose:** The `PercentPopVaccinated` view provides a reusable abstraction to analyze vaccination rates across countries. By summing vaccinations and relating them to population size, the view enables streamlined and consistent reporting.

**Insights Derived:**

1. Comparison of vaccination efforts across countries.
    
2. Countries nearing complete population vaccination.
    
3. A clear overview of global vaccination trends.

In [ ]:
CREATE VIEW PercentPopVaccinated AS
SELECT cd.country, population, SUM(new_vaccinations) AS vaccinated_population
FROM CovidDeaths cd JOIN CovidVaccinations cv
ON cd.country = cv.country AND cd.[date] = cv.[date]
WHERE cd.continent IS NOT NULL
GROUP BY cd.country, population

# Summary of Findings

The SQL data exploration of the COVID-19 dataset revealed key insights:

1. **Pandemic Severity:**
    
    - Countries with the highest death percentages often faced healthcare capacity challenges.
        
    - The affected population percentage highlighted countries with widespread infection rates relative to their population sizes.
        
2. **Vaccination Analysis:**
    
    - Temporary tables and views enabled efficient calculation of vaccinated population percentages.
        
    - Leading countries in vaccination rates were identified, along with regions requiring greater focus.
        
3. **Continental Trends:**
    
    - Aggregated statistics by continent showed variations in COVID-19’s impact and vaccination progress.